In [1]:
import pandas as pd
import re
import json
import matplotlib.pyplot as plt
import gc
from bs4 import BeautifulSoup
from json_stream_parser import load_iter

In [2]:
date = 'mar-26'
path = '/mnt/data/'

In [3]:
# convert html to string
def body2text_body(html):
    soup = BeautifulSoup(html, 'lxml')
    result = []
    # kill all script and style elements
    for script in soup(['script', 'style']):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    for line in lines:
        if line:
            result.append(line)

    return result

In [4]:
LIMIT = 500000
i = 0
data = []

with open(path + 'universe-' + date + '.json') as data_file:
    for obj in load_iter(data_file):
        body = obj['_source']['body']
        body_text = body2text_body(body)
        row = [obj['_source']['url'], obj['_source']['title'], body_text, obj['_source']['status']]
        data.append(row)
        if len(data) >= LIMIT:
            print('Output csv ' + str(i))
            df = pd.DataFrame(data, columns=['url', 'title', 'body_text', 'status'])
            df.to_csv(path + 'universe-'+date+'-'+str(i)+'.csv', index=False)
            del df
            data = []
            i += 1
            gc.collect()

Output csv 0
